In [2]:
from geopy.geocoders import ArcGIS
import pandas as pd
from pyngrok import ngrok
df = pd.read_csv('combined_new.csv')
nom = ArcGIS()

In [3]:
from math import radians, sin, cos, sqrt, atan2
from ipywidgets import widgets, interactive
from IPython.display import display, clear_output
import folium
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert coordinates from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of the Earth in kilometers
    return distance

In [4]:
upshift = pd.read_csv('upshift_locs.csv')

In [5]:
upshift = upshift.rename(columns={'Office Name': 'Customer', 'Address': 'full_addy'})

In [6]:
for _, office_row in upshift.iterrows():
    office_name = office_row['Customer']
    office_latitude = office_row['Latitude']
    office_longitude = office_row['Longitude']
    radius = 60  # Radius in kilometers

    df[office_name] = df.apply(lambda row: 1 if calculate_distance(
        row['Latitude'], row['Longitude'], office_latitude, office_longitude
    ) <= radius else 0, axis=1)

In [7]:
office_dropdown = widgets.Dropdown(
    options=list(upshift['Customer']),
    description='Office Location:'
)

In [8]:
map_object = folium.Map()

In [9]:
def update_map(selected_office):
    selected_office = office_dropdown.value
    if selected_office:
        filtered_df = df[df[selected_office] == 1]
        lat_mean = filtered_df['Latitude'].mean()
        lon_mean = filtered_df['Longitude'].mean()

        new_map = folium.Map(location=[lat_mean, lon_mean], zoom_start=10)

        for _, row in filtered_df.iterrows():
            if selected_office in row['Customer']:
                folium.Circle(
                    location=[row['Latitude'], row['Longitude']],
                    radius=60000,
                    color='green',
                ).add_to(new_map)
                folium.Marker(
                    location=[row['Latitude'], row['Longitude']],
                    tooltip= f"{row['Customer']}: {row['full_addy']}",
                    icon = folium.Icon(color = "green")
                    
                ).add_to(new_map)
                
            elif 'sodexo' in row['Partner'].lower() and row['Current'] == 0:
                folium.Marker(
                    location=[row['Latitude'], row['Longitude']],
                    tooltip=f"{row['Customer']}. Address: {row['full_addy']}. Partner: {row['Partner']}",
                    icon = folium.Icon(color = 'red')
                ).add_to(new_map)
                
            elif 'compass' in row['Partner'].lower() and row['Current'] == 0:
                folium.Marker(
                    location=[row['Latitude'], row['Longitude']],
                    tooltip=f"{row['Customer']}. Address: {row['full_addy']}. Partner: {row['Partner']}",
                    icon = folium.Icon(color = 'blue')
                ).add_to(new_map)
                
            elif 'aramark' in row['Partner'].lower() and row['Current'] == 0:
                folium.Marker(
                    location=[row['Latitude'], row['Longitude']],
                    tooltip=f"{row['Customer']}. Address: {row['full_addy']}. Partner: {row['Partner']}",
                    icon = folium.Icon(color = 'orange')
                ).add_to(new_map)
                
            elif row['Current'] == 1:
                folium.Marker(
                    location=[row['Latitude'], row['Longitude']],
                    tooltip=f"{row['Customer']}. Address: {row['full_addy']}. Partner: {row['Partner']}",
                    icon = folium.Icon(color = 'lightgray')
                ).add_to(new_map)

        clear_output()

        display(new_map)

In [10]:
interactive_map = widgets.interactive(update_map, selected_office=office_dropdown)

In [11]:
interactive_map

interactive(children=(Dropdown(description='Office Location:', options=('UpShift Atlanta', 'UpShift Charlotte'…